In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import os

transformers.logging.set_verbosity_error()

# https://github.com/kakaobrain/KorNLUDatasets
DATA_PATH = 'C:\\Users\\배진우\\Documents\\multiCampus_TA\\python_data\\'
df = pd.read_csv(DATA_PATH + 'multinli.train.koNLI.tsv', delimiter = '\t', quoting = 3)
df = df.dropna()

# gold_label을 수치화한다. [contradiction (모순 관계) = 0, entailment (얽힘) = 1, neutral (중립) = 2]
df['label'] = LabelEncoder().fit_transform(df['gold_label'])

df = df[:100]   # 시험용으로 조금만 사용
df.head()

# DATA_PATH 아래에 모델 파라메터를 저장할 파일을 설정한다.
ckpt_path = DATA_PATH + 'konli_weights.h5'

# Create a callback that saves the model's weights, every epochs
cp_callback = ModelCheckpoint(filepath=ckpt_path, 
                              save_weights_only=True, 
                              verbose=1,
                              save_freq=1)

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)
word2idx = tokenizer.vocab
idx2word = {v:k for k, v in word2idx.items()}

MAX_LEN = 24 * 2

# Bert Tokenizer
def bert_tokenizer_v2(sent1, sent2):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent1,
        text_pair = sent2,
        add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        padding = 'max_length',
        # pad_to_max_length = True,
        return_attention_mask = True,   # Construct attn. masks.
        truncation = True
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids']  # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

text1 = df['sentence1'][0]
text2 = df['sentence2'][0]

id, mask, typ = bert_tokenizer_v2(text1, text2)
print(text1)
print(text2)

print(id)
print(mask)
print(typ)

text_1 = [idx2word[x] for x in id]
print(text_1)

# 문장 복원
print((' '.join(text_1)).replace(' ##', ''))  # 원리
print(tokenizer.decode(id))

print('토큰 길이 =', len(id))

def build_data(sent1, sent2):
    x_ids = []
    x_msk = []
    x_typ = []

    for s1, s2 in tqdm(zip(sent1, sent2), total=len(sent1)):
        input_id, attention_mask, token_type_id = bert_tokenizer_v2(s1, s2)

        x_ids.append(input_id)
        x_msk.append(attention_mask)
        x_typ.append(token_type_id)
    
    x_ids = np.array(x_ids, dtype=int)
    x_msk = np.array(x_msk, dtype=int)
    x_typ = np.array(x_typ, dtype=int)
    # pdb.set_trace()

    return x_ids, x_msk, x_typ

sent1 = df['sentence1']
sent2 = df['sentence2']
y_label = df['label']

x_train1, x_test1, x_train2, x_test2, y_train, y_test = train_test_split(sent1, sent2, y_label, test_size=0.1, random_state = 0)

x_train_ids, x_train_msk, x_train_typ = build_data(x_train1, x_train2)
x_test_ids, x_test_msk, x_test_typ = build_data(x_test1, x_test2)

y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

x_train_ids.shape, y_train.shape, x_test_ids.shape, y_test.shape

x_train_ids[1]

x_train_msk[1]

x_train_typ[1]

bert_model = TFBertModel.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt')
bert_model.summary() # bert_model을 확인한다. trainable params = 177,854,978

# BERT 입력
# ---------
x_input_ids = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)
x_input_msk = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)
x_input_typ = Input(batch_shape = (None, MAX_LEN), dtype = tf.int32)

# BERT 출력
# ---------
output_bert = bert_model([x_input_ids, x_input_msk, x_input_typ])[1]

# Downstream task : 자언어 추론 (NLI)
# ----------------------------------
y_output = Dense(3, activation = 'softmax')(output_bert)
model = Model([x_input_ids, x_input_msk, x_input_typ], y_output)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam(learning_rate = 0.001))
model.summary()

if os.path.exists(ckpt_path):
    model.load_weights(ckpt_path)
    print('학습된 weight가 적용됐습니다.')

x_train = [x_train_ids, x_train_msk, x_train_typ]
x_test = [x_test_ids, x_test_msk, x_test_typ]
hist = model.fit(x_train, y_train, 
                 validation_data = (x_test, y_test), 
                 epochs=10, 
                 batch_size=1024,
                 callbacks=[cp_callback])

# Loss history를 그린다
plt.plot(hist.history['loss'], label='Train loss')
plt.plot(hist.history['val_loss'], label = 'Test loss')
plt.legend()
plt.title("Loss history")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

# 시험 데이터로 학습 성능을 평가한다
pred = model.predict(x_test)
y_pred = np.argmax(pred, axis=1).reshape(-1, 1)
accuracy = (y_pred == y_test).mean()
print("\nAccuracy = %.2f %s" % (accuracy * 100, '%'))


RuntimeError: Failed to import transformers.models.bert because of the following error (look up to see its traceback):
invalid syntax (__init__.py, line 172)